In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

In [156]:
ordini_per_cliente = json.load(open('data/utils/ordini_per_client.json'))
tot_per_cliente = json.load(open('data/utils/tot_per_client.json')) 
quantity = pd.read_csv('data/utils/quantity_per_client.csv')

clienti = list(ordini_per_cliente.keys())

tot = sum(tot_per_cliente.values())
norm = {k: v/tot for k,v in tot_per_cliente.items()}

In [218]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# params
num_samples = 100

# Set to True to generate data starting from today (orders in entrata oggi)
today = True

# Altrimenti imposta un range
start = '2024-10-01'
end = '2024-12-31'

new_data = []

# Loop to generate synthetic data with progress bar
for _ in tqdm(range(num_samples), desc="Generating data"):

    # Scegli cliente in modo uniforme
    i = np.round(np.random.uniform(0, len(clienti) - 1))
    cliente = clienti[int(i)]
    prob = norm[cliente]
    num = int(np.round(prob * num_samples))

    rows = quantity[quantity['Cliente'] == cliente].shape[0]
    if rows > 0:
        articoli = quantity[quantity['Cliente'] == cliente]
        
        for _ in range(num):
            art_index = int(np.round(np.random.uniform(0, rows - 1)))
            art = articoli.iloc[art_index]

            articolo = art['cod articolo']
            quantity_mean = art['mean']
            quantity_std = art['std']
            giorni = art['mean_differenza_giorni']

            # Ensure kg is always positive
            kg = np.round(np.random.normal(quantity_mean, quantity_std))
            if kg <= 0:
                kg = quantity_mean

            if today:
                start = pd.Timestamp.today().date()
            else:
                date_range = pd.date_range(start=start, end=end)
                data_inizio = np.random.choice(date_range)
                data_inizio = pd.Timestamp(data_inizio).date()

            # Calculate the end date
            data_fine = data_inizio + pd.Timedelta(days=giorni)
            data_fine = pd.Timestamp(data_fine).date()

            # Append the generated data to new_data list
            new_data.append({
                'cliente': cliente,
                'cod_articolo': articolo,
                'quantity': kg,
                'data inserimento': data_inizio,
                'data consegna': data_fine
            })

# Create a DataFrame from the new_data list
columns = ['cliente', 'cod_articolo', 'quantity', 'data inserimento', 'data consegna']
res = pd.DataFrame(data=new_data, columns=columns)
res

Generating data: 100%|██████████| 100/100 [00:00<00:00, 1883.38it/s]


,cliente,cod_articolo,quantity,data inserimento,data consegna
0,CSP PARIS,2250ZN,124.0,2024-12-29,2025-01-24
1,CSP PARIS,4466SN1,125.0,2024-12-29,2025-01-24
2,CSP PARIS,4466ZN1,293.0,2024-12-29,2025-02-27
3,W.ZIMMERMANN,4465SN1,84.0,2024-12-29,2025-03-10
4,CALZIFICIO C&LD,2269ZF2,28.0,2024-12-29,2025-01-01
...,...,...,...,...,...
97,DIAMANT,39502DN-14822,22.0,2024-12-29,2025-01-24
98,DIAMANT,94003DN-Q7300,1.0,2024-12-29,2025-01-24
99,DIAMANT,23504DN-Q7175,34.0,2024-12-29,2025-01-24
100,DIAMANT,23504DN-Q5143,245.0,2024-12-29,2025-01-24
